
# TODO

read test files

do normalization separately for fbank and mfcc?

padding

now I first map 48->39. Maybe use 48 for training and map after prediction?

outliers?

In [1]:
import numpy as np

In [2]:
%%time

sentence2frames = dict()

def read_ark(filepath):
    with open(filepath) as train_ark_f:
        last_sentence = ''
        frames = list()
        is_first_line = True
        for line in train_ark_f:
            line = line.strip().split()
            instance_id = line[0]
            speaker_id, sentence_id, frame_id = instance_id.split('_')
            sentence = '_'.join((speaker_id, sentence_id))
            features = list(map(float, line[1:]))

            if sentence != last_sentence and not is_first_line:
                if last_sentence not in sentence2frames:
                    sentence2frames[last_sentence] = frames
                else:
                    sentence2frames[last_sentence] = np.concatenate(
                        (sentence2frames[last_sentence], frames),
                        axis=1
                    )
                frames = list()

            last_sentence = sentence
            frames.append(features)
            is_first_line = False

        # process last sentence
        if last_sentence not in sentence2frames:
            sentence2frames[last_sentence] = frames
        else:
            sentence2frames[last_sentence] = np.concatenate(
                (sentence2frames[last_sentence], frames),
                axis=1
            )
        frames = list()

read_ark('../data/fbank/train.ark')
read_ark('../data/mfcc/train.ark')

CPU times: user 34.5 s, sys: 840 ms, total: 35.4 s
Wall time: 35.4 s


In [3]:
# number of training sentences
print(len(sentence2frames))

# number of frames in this sentence
print(len(sentence2frames['faem0_si1392']))

# number of fbank+mfcc features in a frame
print(len(sentence2frames['faem0_si1392'][0]))

# first feature
print(sentence2frames['faem0_si1392'][0][0])

3696
474
108
3.148541


In [4]:
phone48_phone39 = dict()
phone48_char48 = dict()

with open('../data/phones/48_39.map') as phone48_phone39_f:
    for line in phone48_phone39_f:
        phone48, phone39 = line.strip().split('\t')
        phone48_phone39[phone48] = phone39

with open('../data/48phone_char.map') as phone48_char48_f:
    for line in phone48_char48_f:
        phone48, index, char48 = line.strip().split('\t')
        phone48_char48[phone48] = char48

In [5]:
%%time

sentence2labels = dict()

def read_label(filepath):
    with open(filepath) as train_lab_f:
        last_sentence = ''
        labels = list()
        is_first_line = True
        
        for line in train_lab_f:
            instance_id, label = line.strip().split(',')
            speaker_id, sentence_id, frame_id = instance_id.split('_')
            sentence = '_'.join((speaker_id, sentence_id))
            char = phone48_char48[phone48_phone39[label]]

            if sentence != last_sentence and not is_first_line:
                sentence2labels[last_sentence] = labels
                labels = list()

            last_sentence = sentence
            labels.append(char)
            is_first_line = False

        # process last sentence
        sentence2labels[last_sentence] = labels
        labels = list()

read_label('../data/label/train.lab')

CPU times: user 896 ms, sys: 12 ms, total: 908 ms
Wall time: 877 ms


In [6]:
# number of training sentences
print(len(sentence2labels))

# number of frames in this sentence
print(len(sentence2labels['faem0_si1392']))

# first label
print(sentence2labels['faem0_si1392'][0])

3696
474
L
